In [1]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [2]:
#BEARE TOKEN 
os.environ['TOKEN'] = 'TOKEN'

In [3]:
#CALL THE ENVIRONMENT
def auth():
    return os.getenv('TOKEN')

In [4]:

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [5]:
def create_url(keyword, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

In [6]:

def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [7]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "(#measles OR measles OR monkeypox OR #monkeypox OR monkeypoxvirus OR #monkeypoxvirus OR Norovirus OR #Norovirus OR mumps OR #mumps OR rubella OR #rubella OR winterflu OR #winterflu  OR salmonella OR #salmonella OR shingles OR #shingles OR chickenpox OR #chickenpox OR rabies OR #rabies OR virus OR #virus OR winterbug OR #winterbug OR viralinfection OR #viralinfection OR viral OR #viral OR viralbug OR #viralbug OR flu OR #flu OR infection OR #infection) (place_country:GB) has:geo -is:retweet -is:nullcast lang:en"
start_time = "2020-01-01T00:00:00.000Z"
end_time = "2020-12-31T00:00:00.000Z"
max_results = 500

In [8]:
url = create_url(keyword, start_time,end_time, max_results)

json_response = connect_to_endpoint(url[0], headers, url[1])

Endpoint Response Code: 200


In [9]:
#PRINT JSON file


print(json.dumps(json_response, indent=4, sort_keys=True))

{
    "data": [
        {
            "author_id": "1030190549691760640",
            "conversation_id": "1344426014768844800",
            "created_at": "2020-12-30T23:55:01.000Z",
            "geo": {
                "place_id": "1fb4edc95a98a034"
            },
            "id": "1344431829647060996",
            "in_reply_to_user_id": "1651522832",
            "lang": "en",
            "public_metrics": {
                "like_count": 0,
                "quote_count": 0,
                "reply_count": 0,
                "retweet_count": 0
            },
            "referenced_tweets": [
                {
                    "id": "1344430651043438597",
                    "type": "replied_to"
                }
            ],
            "reply_settings": "everyone",
            "source": "Twitter for Android",
            "text": "@DrDenaGrayson @trvrb I knew it. Higher viral load. Here we go again."
        },
        {
            "author_id": "18722872",
            "conversati

In [10]:
# Create file
csvFile = open("uk20new.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet', 'location','bbox'])
csvFile.close()

In [11]:
#appenson json to CSV

def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        if ('geo' in tweet):   
            geo = tweet['geo']['place_id']
        else:
            geo = " "

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 7. source
        source = tweet['source']

        # 8. Tweet text
        text = tweet['text']
        
        #9. Location
        for place in json_response['includes']['places']:
        
            if (tweet['geo']['place_id']==place['id']):
                location = place['full_name']
                bbox = place ['geo']['bbox']
        
        # Assemble all data in a list
        res = [author_id, created_at, geo, tweet_id, lang, like_count, quote_count, reply_count, retweet_count, source, text, location, bbox]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 

In [12]:
#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "(#measles OR measles OR monkeypox OR #monkeypox OR monkeypoxvirus OR #monkeypoxvirus OR Norovirus OR #Norovirus OR mumps OR #mumps OR rubella OR #rubella OR winterflu OR #winterflu  OR salmonella OR #salmonella OR shingles OR #shingles OR chickenpox OR #chickenpox OR rabies OR #rabies OR virus OR #virus OR winterbug OR #winterbug OR viralinfection OR #viralinfection OR viral OR #viral OR viralbug OR #viralbug OR flu OR #flu OR infection OR #infection) (place_country:GB) has:geo -is:retweet -is:nullcast lang:en"

start_list =    ['2020-01-01T00:00:00.000Z',
                 '2020-02-01T00:00:00.000Z',
                 '2020-03-01T00:00:00.000Z',
                 '2020-04-01T00:00:00.000Z',
                 '2020-05-01T00:00:00.000Z',
                 '2020-06-01T00:00:00.000Z',
                 '2020-07-01T00:00:00.000Z',
                 '2020-08-01T00:00:00.000Z',
                 '2020-09-01T00:00:00.000Z',
                 '2020-10-01T00:00:00.000Z',
                 '2020-11-01T00:00:00.000Z',
                 '2020-12-01T00:00:00.000Z']

end_list =      ['2020-01-31T00:00:00.000Z',
                 '2020-02-29T00:00:00.000Z',
                 '2020-03-31T00:00:00.000Z',
                 '2020-04-30T00:00:00.000Z',
                 '2020-05-31T00:00:00.000Z',
                 '2020-06-30T00:00:00.000Z',
                 '2020-07-31T00:00:00.000Z',
                 '2020-08-31T00:00:00.000Z',
                 '2020-09-30T00:00:00.000Z',
                 '2020-10-31T00:00:00.000Z',
                 '2020-11-30T00:00:00.000Z',
                 '2020-12-31T00:00:00.000Z']
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("uk20new.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 5000 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "uk20new.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "uk20new.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71enieefgn49p3kydw1d9i5mf6gt
Start Date:  2020-01-01T00:00:00.000Z
# of Tweets added from this response:  492
Total # of Tweets added:  492
-------------------
-------------------
Token:  b26v89c19zqg8o3fo71enieefgn49p3kydw1d9i5mf6gt
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71e8ex6g4umpuptcxa26g2v480hp
Start Date:  2020-01-01T00:00:00.000Z
# of Tweets added from this response:  488
Total # of Tweets added:  980
-------------------
-------------------
Token:  b26v89c19zqg8o3fo71e8ex6g4umpuptcxa26g2v480hp
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71dtdkzs8up2t261injm3j3o3531
Start Date:  2020-01-01T00:00:00.000Z
# of Tweets added from this response:  491
Total # of Tweets added:  1471
-------------------
-------------------
Token:  b26v89c19zqg8o3fo71dtdkzs8up2t261injm3j3o3531
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6yhtrsf6olh4gbvaqavfuyspqk59
Sta